In [23]:
import os
import sys
os.chdir("/Users/DanPfeiffer/Documents/code/data-collector/src")
sys.path.append(os.getcwd())
import pandas as pd
import numpy as np
from feature_eng import matchup_creator, box_score_feature_adder
import importlib
from datetime import datetime
from preprocess import preprocess_tr_stats, preprocess_box_scores
import json
import plotly.express as px

importlib.reload(matchup_creator)
importlib.reload(preprocess_tr_stats)
importlib.reload(preprocess_box_scores)
importlib.reload(box_score_feature_adder)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# stat_df_raw = pd.read_excel("../data/raw/full_stats.xlsx")

In [4]:
stat_df = stat_df_raw.copy()

In [5]:
stat_df.shape

(20128, 1589)

In [6]:
stat_df = preprocess_tr_stats.preprocess(stat_df)

In [7]:
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20128 entries, 0 to 20127
Columns: 215 entries, date to penalties_opponent_penalty_rate
dtypes: datetime64[ns, UTC](1), float64(213), object(1)
memory usage: 33.0+ MB


In [8]:
box_df_raw = pd.read_csv("../data/raw/box_scores_df.csv")
box_df_raw.shape

(6265, 72)

In [9]:
box_df = box_df_raw.copy()

In [10]:
box_df = preprocess_box_scores.preprocess(box_df)

In [11]:
box_processed = box_df.copy()

In [12]:
bsfa = box_score_feature_adder.BoxScoreFeatureAdder(box_df=box_processed)

In [13]:
box_scores_df = bsfa.add_features(df=box_processed)

adding location info
adding location info
adding location info
adding local time
adding travel distance
adding home travel
adding away travel
adding days rest
adding box totals
adding spread results
adding totals results


/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:112: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{prefix}{spread}_home_cover"] = np.where(
/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:112: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{prefix}{spread}_home_cover"] = np.where(
/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:112: PerformanceWarning: DataFrame is highly fragmented.  This

In [14]:
box_scores_df.shape

(5128, 287)

In [15]:
post_processed_box_df= box_scores_df.copy().sort_values(by="date", ascending=False)

In [16]:
def generate_features(row, creator):
    game_id = row['id']
    home_short_display_name = row['home_short_display_name']
    away_short_display_name = row['away_short_display_name'] 
    date = row['date']
    features_df = creator.create_matchup(game_id=game_id, 
                                         home_team_box_short_display_name=home_short_display_name, 
                                         away_team_box_short_display_name=away_short_display_name, 
                                         date=date)
    return features_df.iloc[0]

In [27]:
importlib.reload(matchup_creator)

for games_to_sample in np.arange(start=15, stop=9, step=-1):
    decay_factor = 1.0
    agg_method = "exp_weighted_mean"
    experiment_name = f"{games_to_sample}_games_{agg_method}_decay{decay_factor}"
    print()
    print(experiment_name)
    print()
    mc = matchup_creator.MatchupCreator(stats_db=stat_df,
                                        games_to_sample=games_to_sample, 
                                        aggregation_method=agg_method,
                                        decay_factor=decay_factor)
    features_df = post_processed_box_df.apply(lambda row: generate_features(row, mc), axis=1)
    df_final = pd.merge(post_processed_box_df, features_df, on='game_id', how='left')
    df_final.to_csv(f'../data/transformed/{experiment_name}_training.csv', index=False)

for decay_factor in [0.1, 0.85]:
    agg_method = "inv_log_weighted_mean"
    for games_to_sample in np.arange(start=16, stop=9, step=-1):
        experiment_name = f"{games_to_sample}_games_{agg_method}_decay{decay_factor}"
        print()
        print(experiment_name)
        print()
        mc = matchup_creator.MatchupCreator(stats_db=stat_df,
                                            games_to_sample=games_to_sample, 
                                            aggregation_method=agg_method,
                                            decay_factor=decay_factor)
        features_df = post_processed_box_df.apply(lambda row: generate_features(row, mc), axis=1)
        df_final = pd.merge(post_processed_box_df, features_df, on='game_id', how='left')
        df_final.to_csv(f'../data/transformed/{experiment_name}_training.csv', index=False)


15_games_exp_weighted_mean_decay1.0

creating matchups for 2024
creating matchups for 2023
creating matchups for 2022
creating matchups for 2021
creating matchups for 2020
creating matchups for 2019
creating matchups for 2018
creating matchups for 2017
creating matchups for 2016
creating matchups for 2015
creating matchups for 2014
creating matchups for 2013
creating matchups for 2012
creating matchups for 2011
creating matchups for 2010
creating matchups for 2009
creating matchups for 2008
creating matchups for 2007
creating matchups for 2006
creating matchups for 2005

14_games_exp_weighted_mean_decay1.0

creating matchups for 2024
creating matchups for 2023
creating matchups for 2022
creating matchups for 2021
creating matchups for 2020
creating matchups for 2019
creating matchups for 2018
creating matchups for 2017
creating matchups for 2016
creating matchups for 2015
creating matchups for 2014
creating matchups for 2013
creating matchups for 2012
creating matchups for 2011
creati

In [ ]:
offense_scoring_points_per_game
offense_scoring_yards_per_point
offense_scoring_points_per_play
defense_scoring_ppg
defense_scoring_yards_per_point
defense_scoring_points_per_play

In [28]:
df_final.shape

(5128, 915)